<a href="https://colab.research.google.com/github/Swaraj-CSE-AI/My-ML-Project/blob/main/NLP_playGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras
from keras.preprocessing import sequence
import numpy as np
import pandas as pd
import os

Dataset

In [2]:
path_to_file = keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Load own file

In [3]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

In [4]:
text  = open (path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} charecters'.format(len(text)))

Length of text: 1115394 charecters


In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Encoding

In [6]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])
text_as_int = text_to_int(text)

In [7]:
print("text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [8]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


Training Examples

In [9]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [11]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Training Batches

In [13]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Model Building

In [14]:
#Option 1: Sequential API (simplest, stateless LSTM)
#If you don’t need stateful=True, just use Sequential cleanly:

# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape=(None,)),
#         tf.keras.layers.LSTM(rnn_units,
#                              return_sequences=True,
#                              recurrent_initializer='glorot_uniform'),
#         tf.keras.layers.Dense(vocab_size)
#     ])
#     return model

# model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
# model.summary()


# Option 2: Functional API (needed if stateful=True)
# If you must use a stateful LSTM, build with the functional API:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    inputs = tf.keras.layers.Input(batch_shape=(batch_size, None))
    x = tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    x = tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform')(x)
    outputs = tf.keras.layers.Dense(vocab_size)(x)

    return tf.keras.Model(inputs, outputs)

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (64, None)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (64, None, 256)        │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (64, None, 1024)       │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, None, 65)         │        66,625 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,330,241 (20.33 MB)

 Trainable params: 5,330,241 (20.33 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [16]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 0.00786513  0.0062862  -0.00371203 ... -0.00442869  0.00371211
   -0.00366103]
  [ 0.00491013  0.00668259 -0.00766491 ... -0.00464759  0.00495248
   -0.00640829]
  [ 0.00296024  0.00688455 -0.01072411 ... -0.00522768  0.00562367
   -0.0088132 ]
  ...
  [ 0.00041989 -0.00219758 -0.01073858 ...  0.00415038  0.0083595
   -0.01596026]
  [ 0.00304538 -0.00376219 -0.01279312 ...  0.00561359  0.00517232
   -0.00985358]
  [ 0.01030111  0.00290691 -0.01395492 ...  0.00083042  0.00638521
   -0.01184054]]

 [[ 0.00699978  0.00165385  0.00037607 ...  0.00061798 -0.00382203
   -0.00363508]
  [-0.00206175  0.00193184 -0.005914   ...  0.00069749  0.00066895
   -0.00078589]
  [ 0.00241973  0.0003699   0.00279437 ...  0.0004007   0.00529348
    0.00137515]
  ...
  [ 0.0033033  -0.01111963 -0.00911675 ...  0.00898646  0.0020718
   -0.01173422]
  [-0.00272089 -0.00770405 -0.00673998 ...  0.00071324  0.00324975
   -0.00846773]
  [ 0.00262563 -0.00383085 -0.00681848 ... -0.00326883  0.003

In [17]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[ 0.00786513  0.0062862  -0.00371203 ... -0.00442869  0.00371211
  -0.00366103]
 [ 0.00491013  0.00668259 -0.00766491 ... -0.00464759  0.00495248
  -0.00640829]
 [ 0.00296024  0.00688455 -0.01072411 ... -0.00522768  0.00562367
  -0.0088132 ]
 ...
 [ 0.00041989 -0.00219758 -0.01073858 ...  0.00415038  0.0083595
  -0.01596026]
 [ 0.00304538 -0.00376219 -0.01279312 ...  0.00561359  0.00517232
  -0.00985358]
 [ 0.01030111  0.00290691 -0.01395492 ...  0.00083042  0.00638521
  -0.01184054]], shape=(100, 65), dtype=float32)


In [18]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[ 0.00786513  0.0062862  -0.00371203  0.00426101 -0.00266908  0.00386203
 -0.00190128 -0.00107597 -0.00167902 -0.00160444 -0.00054469 -0.00418652
 -0.00261001  0.00055958 -0.00228554 -0.00132797 -0.00255626  0.00342881
 -0.00225848  0.00381575  0.00068623 -0.00242871 -0.00099379 -0.00092308
  0.00526732  0.0011703  -0.00054854 -0.00263492  0.00047431  0.00332117
  0.00458129  0.00202945 -0.00519435  0.00124015 -0.00315216 -0.004338
 -0.00326751 -0.0039294  -0.00366595  0.00077512 -0.00354914 -0.00784563
 -0.00465124  0.00364712 -0.0024905   0.00297591  0.00315593 -0.00080676
  0.00607526  0.00935796 -0.00215229  0.00264851 -0.00023754  0.00159578
  0.00281221  0.00037538 -0.00506521 -0.00629953  0.00035914  0.00095056
 -0.00077588  0.00402978 -0.00442869  0.00371211 -0.00366103], shape=(65,), dtype=float32)


In [19]:
sampled_indices =  tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

'EoA:&nDSRwrgs:TYP&UQmNMkPQr;EXlfEZtjmJM!eB&H&DZu ?WfBLeAnemr Z&mHirlN!BIlJlp-PeuEqxg3WHEcmV.3fU&\n,lF'

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

Compile and Checkpoints

In [21]:
model.compile(optimizer='adam', loss=loss)

In [22]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix =  os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Training and Loading the Model

In [24]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 68ms/step - loss: 2.6838
Epoch 2/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 69ms/step - loss: 1.8099
Epoch 3/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 71ms/step - loss: 1.5665
Epoch 4/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - loss: 1.4492
Epoch 5/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 74ms/step - loss: 1.3845
Epoch 6/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - loss: 1.3311
Epoch 7/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 78ms/step - loss: 1.2934
Epoch 8/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - loss: 1.2565
Epoch 9/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 79ms/step - loss: 1.2232
Epoch 10/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - loss: 1.1895
Epoch 11/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - loss: 1.1565
Epoch 12/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 79ms/step - loss: 1.1213
Epoch 13/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 78ms/step - loss: 1.0879
Epoch 14/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 77ms/step - loss: 1.0524
Epoch 15/50
172/172 ━━━━━━━━━

In [25]:
import os
print(os.listdir(checkpoint_dir))

['ckpt_35.weights.h5', 'ckpt_33.weights.h5', 'ckpt_1.weights.h5', 'ckpt_25.weights.h5', 'ckpt_20.weights.h5', 'ckpt_29.weights.h5', 'ckpt_6.weights.h5', 'ckpt_13.weights.h5', 'ckpt_41.weights.h5', 'ckpt_27.weights.h5', 'ckpt_46.weights.h5', 'ckpt_37.weights.h5', 'ckpt_49.weights.h5', 'ckpt_2.weights.h5', 'ckpt_32.weights.h5', 'ckpt_18.weights.h5', 'ckpt_21.weights.h5', 'ckpt_14.weights.h5', 'ckpt_10.weights.h5', 'ckpt_9.weights.h5', 'ckpt_15.weights.h5', 'ckpt_16.weights.h5', 'ckpt_44.weights.h5', 'ckpt_50.weights.h5', 'ckpt_30.weights.h5', 'ckpt_23.weights.h5', 'ckpt_42.weights.h5', 'ckpt_34.weights.h5', 'ckpt_17.weights.h5', 'ckpt_7.weights.h5', 'ckpt_31.weights.h5', 'ckpt_24.weights.h5', 'ckpt_8.weights.h5', 'ckpt_45.weights.h5', 'ckpt_38.weights.h5', 'ckpt_28.weights.h5', 'ckpt_11.weights.h5', 'ckpt_43.weights.h5', 'ckpt_40.weights.h5', 'ckpt_19.weights.h5', 'ckpt_39.weights.h5', 'ckpt_26.weights.h5', 'ckpt_12.weights.h5', 'ckpt_5.weights.h5', 'ckpt_36.weights.h5', 'ckpt_4.weights.

In [26]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [29]:
latest_checkpoint_path = os.path.join(checkpoint_dir, "ckpt_50.weights.h5")

model.load_weights(latest_checkpoint_path)
model.build(tf.TensorShape([1, None]))

In [ ]:
#To check if checkpoint exists
# import os

# checkpoint_path = os.path.join(checkpoint_dir, "ckpt_2.weights.h5")

# if os.path.exists(checkpoint_path):
#     model.load_weights(checkpoint_path)
#     model.build(tf.TensorShape([1, None]))
#     print(f"✅ Loaded weights from {checkpoint_path}")
# else:
#     print("⚠️ Checkpoint for epoch 10 not found.")


✅ Loaded weights from ./training_checkpoints/ckpt_2.weights.h5


In [31]:
checkpoint_path = os.path.join(checkpoint_dir, "ckpt_33.weights.h5")

model.load_weights(checkpoint_path)
model.build(tf.TensorShape([1, None]))

print(f"✅ Loaded weights from {checkpoint_path}")


✅ Loaded weights from ./training_checkpoints/ckpt_33.weights.h5


Generating Text

In [32]:
# from inspect import ismemberdescriptor
def generate_text(model, start_string):
  num_generate = 800

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []
  temperature =  1.0

  # model.reset_states() # This is not the correct way to reset states for a functional model,
  #only for ststeful=True RNN model
  for i in range(num_generate):

    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions/temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
inp = input("Type a string starting: ")
print(generate_text(model, inp))

Type a string starting: Romeo
Romeo.

NORTHUMBERLAND:
Read no more, I did: it is a man
To chamber where he shall attend and s
e tongues? O myself,
Itake on our channel service
Being warm'd with the coffence? Speak not for strong,
And all the promisues of his son took somewhat is happines;
De
Contain as the last, I emp supdosed hath tune his hands.

WARWICK:
Then I degrans thee, I, brow thee a person. say, where I am like too late!

GLOUCESTER:
Sweet pray,
Her dukedom to of gring me
But one of nevereig to the people mock'd.

SICINIUS:
Knoce I am an hundred breth
To give him anon.

ANGELO:
Go wain, old Gaunt, take you you.

LARTIUS:
Hark, Perditan me a weardran. What
was e here will I appear,
Accompany more four ancount of worship, breasts, invisube vouch,
To die for your title to them;
And more, more love.

GEONA:
Why, Camil
